In [4]:
%load_ext autoreload

%autoreload 2

In [1]:
import pandas as pd

In [51]:
events_df = pd.read_csv('anchors.csv')

In [52]:
from beads.simple import SimpleExecutor

In [53]:
from beads.composed import ComposedExecutor

In [54]:
def time_to_seconds(time_str):
    hours, minutes, seconds = map(int, time_str.split(":"))
    return hours * 3600 + minutes * 60 + seconds

In [55]:
events_df['Start_time'] = events_df['Start'].apply(lambda x: time_to_seconds(x))
events_df['End_time'] = events_df['End'].apply(lambda x: time_to_seconds(x))

In [56]:
query_events_derivative_df = events_df[events_df['Type'] == 'derivative'].rename(columns={
    'Participant': 'user_id',
    'Feature': 'type',
    'Start_time': 'time',
})[['user_id', 'type', 'time']]

In [130]:
query_events_non_derivative_starts_df = events_df[events_df['Type'] == 'non-derivative'].rename(columns={
    'Participant': 'user_id',
    'Feature': 'type',
    'Start_time': 'time',
})[['user_id', 'type', 'time']]

In [131]:
query_events_non_derivative_starts_df['type'] += '_start'

In [132]:
query_events_non_derivative_starts_df['start_id'] = query_events_non_derivative_starts_df.index

In [133]:
query_events_non_derivative_ends_df = events_df[events_df['Type'] == 'non-derivative'].rename(columns={
    'Participant': 'user_id',
    'Feature': 'type',
    'End_time': 'time',
})[['user_id', 'type', 'time']]

In [134]:
query_events_non_derivative_ends_df['end_id'] = query_events_non_derivative_starts_df.index

In [135]:
query_events_non_derivative_ends_df['type'] += '_end'

In [150]:
query_events_df = pd.concat([
    query_events_derivative_df,
    query_events_non_derivative_starts_df,
    query_events_non_derivative_ends_df
]).reset_index(drop=True)

In [151]:
# query_events_df

In [152]:
query_events_df['session_id'] = query_events_df['user_id']

In [153]:
composed = ComposedExecutor(query_events_df)

In [154]:
composed.query_events_df

,user_id,type,time,start_id,end_id,session_id,event_order_id
event_index,,,,,,,
144-1_0,144-1,round_1_start,922,1.0,NaN,144-1,0
144-1_1,144-1,round_start,922,0.0,NaN,144-1,1
144-1_2,144-1,All player_died after round 1_start,1034,177.0,NaN,144-1,2
144-1_3,144-1,All player_died after round 1_start,1034,180.0,NaN,144-1,3
144-1_4,144-1,exploration_phase_1_start,1034,19.0,NaN,144-1,4
...,...,...,...,...,...,...,...
391-2_493,391-2,camp_combat_end,3231,NaN,4498.0,391-2,493
391-2_494,391-2,main_quest_camp_end,3231,NaN,4501.0,391-2,494
391-2_495,391-2,round_3_end,3231,NaN,4272.0,391-2,495


In [155]:
simple = SimpleExecutor(composed.query_events_df)

In [156]:
# query = [
#     {
#         'node_conditions': ['type == "exploration_phase_start"'],
#     },
#     {
#         'node_conditions': ['type == "camp_combat_start"']
#     },
#     {
#         'node_conditions': ['type == "camp_combat_start"'],
#     },
#     {
#         'node_conditions': ['type == "exploration_phase_end"'],
#         'order_conditions': [{
#             'node_used': 0,
#             'condition': 'start_id_A == end_id_B'
#         }]
#     }
# ]

In [164]:
query = [
    {
        'node_conditions': ['type == "round_3_start"'],
    },
    {
        'node_conditions': ['type == "camp_combat_start"']
    },
    {
        'node_conditions': ['type == "camp_combat_end"'],
        'order_conditions': [{
            'node_used': 1,
            'condition': 'start_id_A == end_id_B'
        }]
    },
    {
        'node_conditions': ['type == "round_3_end"'],
        'order_conditions': [{
            'node_used': 0,
            'condition': 'start_id_A == end_id_B'
        }]
    }
]

In [169]:
camp_combats_df = simple.execute(query)

Step 1 initiated
Step 1 finished
Step 2 initiated
Step 2 finished
Step 3 initiated
Step 3 finished


In [186]:
for i, row in camp_combats_df[
    camp_combats_df['user_id'] == '292-2'
].iterrows():
    print(test_time[row['event_1']], test_time[row['event_2']])

2683 2785
2683 2785
2711 2785
2711 2785
2814 3032
2814 3032
2850 3032
2850 3032
2895 3032
2895 3032
3026 3032
3026 3032


(2683, 2785)

In [180]:
test_time = composed.query_events_df[
    (composed.query_events_df['user_id'] == '292-2') 
].set_index('event_order_id')['time'].to_dict()

In [168]:
.groupby('user_id').size()

Step 1 initiated
Step 1 finished
Step 2 initiated
Step 2 finished
Step 3 initiated
Step 3 finished


user_id
144-1     5
144-2     7
153-2    20
155-2    11
274-2    10
292-2    12
345-2    14
388-2     3
391-2    19
dtype: int64

In [161]:
composed.query_events_df[
    (composed.query_events_df['user_id'] == '144-1') &\
    (composed.query_events_df['event_order_id'] >= 17) &\
    (composed.query_events_df['event_order_id'] <= 226) 
]

,user_id,type,time,start_id,end_id,session_id,event_order_id
event_index,,,,,,,
144-1_17,144-1,All camp_combat during round 1_end,1107,NaN,204.0,144-1,17
144-1_18,144-1,All camp_combat during round 1_start,1107,206.0,NaN,144-1,18
144-1_19,144-1,alv_camp_start,1113,28.0,NaN,144-1,19
144-1_20,144-1,camp_combat_1_start,1113,26.0,NaN,144-1,20
144-1_21,144-1,camp_combat_T0_start,1113,27.0,NaN,144-1,21
...,...,...,...,...,...,...,...
144-1_222,144-1,bandit_camp_start,2115,100.0,NaN,144-1,222
144-1_223,144-1,camp_combat_1_start,2115,98.0,NaN,144-1,223
144-1_224,144-1,camp_combat_T1_start,2115,99.0,NaN,144-1,224


In [163]:
events_df[
    (events_df['Participant'] == '144-1') & \
    (events_df['Feature'] == 'camp_combat')
]

,Participant,Feature,Start,End,Type,Start_time,End_time
21,144-1,camp_combat,00:18:17,00:18:57,non-derivative,1097,1137
25,144-1,camp_combat,00:18:33,00:18:57,non-derivative,1113,1137
29,144-1,camp_combat,00:18:37,00:18:57,non-derivative,1117,1137
39,144-1,camp_combat,00:20:39,00:21:54,non-derivative,1239,1314
46,144-1,camp_combat,00:21:56,00:22:10,non-derivative,1316,1330
56,144-1,camp_combat,00:25:33,00:26:38,non-derivative,1533,1598
63,144-1,camp_combat,00:27:57,00:29:14,non-derivative,1677,1754
76,144-1,camp_combat,00:30:58,00:31:35,non-derivative,1858,1895
86,144-1,camp_combat,00:34:18,00:34:56,non-derivative,2058,2096
90,144-1,camp_combat,00:34:45,00:34:56,non-derivative,2085,2096


In [117]:
composed.query_events_df[
    (composed.query_events_df['user_id'] == '144-1') &\
    (composed.query_events_df['type'] == 'round_3_end') 
]

,user_id,type,time,start_id,end_id,session_id,event_order_id
event_index,,,,,,,
144-1_220,144-1,round_3_end,3579,NaN,17.0,144-1,220


In [113]:
query_events_df.query('type == "round_3_end"')

,user_id,type,time,start_id,end_id,session_id
4527,144-1,round_3_end,3579,NaN,17.0,144-1
4718,144-2,round_3_end,672,NaN,208.0,144-2
4979,153-2,round_3_end,2379,NaN,477.0,153-2
5544,155-2,round_3_end,2893,NaN,1044.0,155-2
6444,274-2,round_3_end,2668,NaN,1950.0,274-2
6833,292-2,round_3_end,3616,NaN,2341.0,292-2
7775,345-2,round_3_end,3416,NaN,3289.0,345-2
8143,388-2,round_3_end,1267,NaN,3659.0,388-2
8750,391-2,round_3_end,3231,NaN,4272.0,391-2


Много времени тратим на перебор ненужных вариантов в случае, если отрезок уже был выбран
- Первым запросом найти вершины начала и вершины конца
- 